In [1]:
import random
import math

class TicTacToe:
    def __init__(self):
        self.board = [' ' for _ in range(9)]  # 3x3 Tic-Tac-Toe board
        self.current_player = 'X'  # X always starts

    def make_move(self, position):
        if self.board[position] == ' ':
            self.board[position] = self.current_player
            self.current_player = 'O' if self.current_player == 'X' else 'X'
            return True
        return False

    def get_possible_moves(self):
        return [i for i, x in enumerate(self.board) if x == ' ']

    def is_winner(self, player):
        winning_positions = [(0,1,2), (3,4,5), (6,7,8), (0,3,6), (1,4,7), (2,5,8), (0,4,8), (2,4,6)]
        return any(all(self.board[pos] == player for pos in combo) for combo in winning_positions)

    def is_draw(self):
        return ' ' not in self.board

    def get_winner(self):
        for player in ['X', 'O']:
            if self.is_winner(player):
                return player
        return None

    def clone(self):
        clone = TicTacToe()
        clone.board = self.board[:]
        clone.current_player = self.current_player
        return clone

class Node:
    def __init__(self, game_state, parent=None, move=None):
        self.game_state = game_state
        self.parent = parent
        self.move = move
        self.wins = 0
        self.visits = 0
        self.children = []
        self.untried_moves = game_state.get_possible_moves()

    def expand(self):
        move = self.untried_moves.pop()
        new_game_state = self.game_state.clone()
        new_game_state.make_move(move)
        child_node = Node(new_game_state, parent=self, move=move)
        self.children.append(child_node)
        return child_node

    def update(self, result):
        self.visits += 1
        if result == self.game_state.current_player:
            self.wins += 1

    # def best_child(self):
    #     return max(self.children, key=lambda child: child.wins / child.visits)
    
    # UCT
    def best_child(self, c_param=math.sqrt(2)):
        choices_weights = [
            (child.wins / child.visits) + c_param * math.sqrt((2 * math.log(self.visits) / child.visits))
            for child in self.children
        ]
        return self.children[choices_weights.index(max(choices_weights))]
    
    def get_children_win_rates(self):
        win_rates = {}
        for child in self.children:
            if child.visits > 0:
                win_rates[child.move] = child.wins / child.visits
            else:
                win_rates[child.move] = 0
        return win_rates

def visualize_action_values(game):
    mcts_root = Node(game.clone())
    _ = mcts(mcts_root, iterations=1000)  # Run MCTS
    win_rates = mcts_root.get_children_win_rates()
    for move, rate in win_rates.items():
        print(f"Move: {move}, Estimated Win Rate: {rate:.2f}")
        
def minimax(game_state, depth, is_maximizing_player):
    if game_state.is_winner('X'):
        return 1
    elif game_state.is_winner('O'):
        return -1
    elif game_state.is_draw():
        return 0

    if is_maximizing_player:
        best_val = -float('inf')
        for move in game_state.get_possible_moves():
            game_state_clone = game_state.clone()
            game_state_clone.make_move(move)
            value = minimax(game_state_clone, depth - 1, False)
            best_val = max(best_val, value)
        return best_val
    else:
        best_val = float('inf')
        for move in game_state.get_possible_moves():
            game_state_clone = game_state.clone()
            game_state_clone.make_move(move)
            value = minimax(game_state_clone, depth - 1, True)
            best_val = min(best_val, value)
        return best_val

def simulate_random_playout(game_state):
    if len(game_state.get_possible_moves()) <= 4:
        return 'X' if minimax(game_state, depth=10, is_maximizing_player=False) > 0 else 'O'
    else:
        while True:
            possible_moves = game_state.get_possible_moves()
            if not possible_moves:
                break
            game_state.make_move(random.choice(possible_moves))
            if game_state.is_winner(game_state.current_player) or game_state.is_draw():
                break
        return game_state.get_winner()
    
    # while True:
    #     possible_moves = game_state.get_possible_moves()
    #     if not possible_moves:
    #         break
    #     game_state.make_move(random.choice(possible_moves))
    #     if game_state.is_winner(game_state.current_player) or game_state.is_draw():
    #         break
    # return game_state.get_winner()

def mcts(root, iterations=1000):
    for _ in range(iterations):
        node = root
        game_state = root.game_state.clone()

        # Selection
        while node.untried_moves == [] and node.children != []:
            node = node.best_child()
            game_state.make_move(node.move)

        # Expansion
        if node.untried_moves != []:
            node = node.expand()
            game_state.make_move(node.move)

        # Simulation
        winner = simulate_random_playout(game_state)

        # Backpropagation
        while node is not None:
            node.update(winner)
            node = node.parent

    return root.best_child().move

def visualize_board(board):
    # Format the board into a 3x3 grid
    rows = [board[i:i+3] for i in range(0, len(board), 3)]
    board_str = '\n'.join([' | '.join(row) for row in rows])
    board_str = board_str.replace(' ', '_')
    return board_str

# Using the MCTS for a Tic-Tac-Toe game
# game = TicTacToe()
# mcts_root = Node(game)
# best_move = mcts(mcts_root, iterations=1000)
# best_move
# game.make_move(best_move)


    
def player_vs_mcts(game, mcts_player='O'):
    while True:
        # Human player's turn
        if game.current_player == 'X':
            print("Current board:")
            print(visualize_board(game.board))
            move = int(input("Your move (0-8): "))
            if not (0 <= move <= 8) or not game.make_move(move):
                print("Invalid move, try again.")
                continue

        # MCTS's turn
        else:
            print("MCTS is thinking...")
            mcts_root = Node(game.clone())
            mcts_move = mcts(mcts_root, iterations=1000)
            game.make_move(mcts_move)
            print(f"MCTS played at position {mcts_move}")

            # Visualize the estimated value of each action
            action_values = mcts_root.get_children_win_rates()
            print("Estimated action values:")
            for move, value in action_values.items():
                print(f"Move {move}: Win Rate {value:.2f}")
                
        # Check for end of game
        winner = game.get_winner()
        if winner or game.is_draw():
            print("Final board:")
            print(visualize_board(game.board))
            if winner:
                print(f"Player '{winner}' wins!")
            else:
                print("It's a draw!")
            break

# Create a new game and start the loop
new_game = TicTacToe()
player_vs_mcts(new_game)




Current board:
__|___|__
__|___|__
__|___|__
MCTS is thinking...
MCTS played at position 7
Estimated action values:
Move 8: Win Rate 0.45
Move 7: Win Rate 0.60
Move 6: Win Rate 0.48
Move 5: Win Rate 0.57
Move 4: Win Rate 0.42
Move 3: Win Rate 0.51
Move 2: Win Rate 0.44
Move 1: Win Rate 0.57
Current board:
X_|___|__
__|___|__
__|_O_|__
MCTS is thinking...
MCTS played at position 5
Estimated action values:
Move 6: Win Rate 0.85
Move 5: Win Rate 0.93
Move 4: Win Rate 0.24
Move 3: Win Rate 0.62
Move 2: Win Rate 0.62
Move 1: Win Rate 0.56
Current board:
X_|___|__
__|___|_O
__|_O_|_X
MCTS is thinking...
MCTS played at position 4
Estimated action values:
Move 6: Win Rate 1.00
Move 4: Win Rate 0.97
Move 3: Win Rate 0.99
Move 1: Win Rate 0.98
Current board:
X_|___|_X
__|_O_|_O
__|_O_|_X
MCTS is thinking...
MCTS played at position 3
Estimated action values:
Move 3: Win Rate 1.00
Move 1: Win Rate 1.00
Final board:
X_|___|_X
O_|_O_|_O
X_|_O_|_X
Player 'O' wins!
